# **SISTEMA BASADO EN REGLAS**

In [ ]:
from datetime import datetime, timedelta

In [ ]:
def hora_en_franjas(hora: str, franjas: list[tuple[str, str]], step_minutes: int = 15) -> bool:
  #step_minutes = 15 recorre las franjas horarias cada 15 minutos.

    formato = "%H:%M"
    hora_obj = datetime.strptime(hora, formato)
    for inicio, fin in franjas:
        t_inicio = datetime.strptime(inicio, formato)
        t_fin = datetime.strptime(fin, formato)
        while t_inicio < t_fin:
            if t_inicio == hora_obj:
                return True
            t_inicio += timedelta(minutes=step_minutes)
    return False

## SALA 7.MAMOGRAFÍA

### BAV por ETX

In [ ]:
def es_horario_bav_por_etx(hora: str, dia: str) -> bool:
    """Citas BAV por ETX:
    - Viernes: 08:45, 10:15, 11:15, 12:15
    - Lunes: 13:00
    """
    dia = dia.lower()
    if dia == "viernes":
        return hora in ["08:45", "10:15", "11:15", "12:15"]
    elif dia == "lunes":
        return hora == "13:00"
    else:
        return False

### Arpón por ETX

In [ ]:
def es_horario_arpon(hora: str, dia: str) -> bool:
    """Arpones:
    - Solo los martes y jueves
    - Citas de 45 minutos entre 08:30 y 09:45
    """
    dia = dia.lower()
    if dia not in ["martes", "jueves"]:
        return False
    return hora in ["08:30", "09:15"]

### Tomografías estándar

Además de los huecos destinados a realizar tomografías, se tienen en cuenta las siguientes condiciones para la citación:
Se añadirán huecos de tomografía de 15 minutos SI:
- los lunes no hay una BAV de 13:00 a 14:00.
- los viernes no hay BAV.

In [ ]:
def es_horario_tx_mamografia(hora: str, dia: str, ocupado_bav: bool = False, ocupado_arpon: bool = False) -> bool:
    dia = dia.lower()
    franjas = []

    if dia in ["lunes", "viernes"]:
        franjas = [("08:30", "10:45"), ("11:45", "13:15")]

        if dia == "lunes":
            if not ocupado_bav:
                franjas.append(("13:15", "13:45"))

        elif dia == "viernes":
            franjas.append(("13:15", "13:45"))

            if ocupado_bav:
                # Excluir franjas BAV por ETX del viernes: ["08:45", "10:15", "11:15", "12:15"]
                franjas_bav = [("08:45", "09:45"), ("10:15", "11:15"),
                               ("11:15", "12:15"), ("12:15", "13:15")]

                # Filtramos franjas para eliminar solapamientos con BAV
                def sin_solape(f1, f2):
                    return f1[1] <= f2[0] or f1[0] >= f2[1]

                franjas = [f for f in franjas
                           if all(sin_solape(f, b) for b in franjas_bav)]

    elif dia in ["martes", "jueves"]:
        if not ocupado_arpon:
            franjas = [("08:30", "10:45")]
        else:
            franjas = [("10:00", "10:45")]
        franjas.append(("12:00", "13:00"))

    return hora_en_franjas(hora, franjas, step_minutes=15)

### Mamografía con contraste

In [ ]:
def es_horario_mamografia_con_contraste(hora: str, dia: str) -> bool:
    """
    Mamografía con contraste:
    - Solo miércoles
    - Citas de 45 minutos desde 08:45 a 13:45
    """
    dia = dia.lower()
    if dia != "miércoles":
        return False

    inicio = datetime.strptime("08:45", "%H:%M")
    fin = datetime.strptime("13:45", "%H:%M")

    while inicio < fin:
        if inicio.strftime("%H:%M") == hora:
            return True
        inicio += timedelta(minutes=45)

    return False

## S7.ECOGRAFÍA

### Ecografías diagnósticas: prioritaria, ordinaria, revaloración y +contraste

In [ ]:
def es_horario_ecografia_s7(hora: str, dia: str, tipo: str) -> bool:
    tipo = tipo.lower()
    dia = dia.lower()

    if dia == "lunes":
        if tipo in ["revaloracion", "contraste"]:
            return hora_en_franjas(hora, [("09:00", "13:45")], step_minutes=30)
        elif tipo == "ordinaria":
            return hora_en_franjas(hora, [("09:00", "13:45")], step_minutes=15)

    elif dia in ["martes", "jueves"]:
        if tipo == "prioritaria":
            return hora_en_franjas(hora, [("09:00", "09:30")], step_minutes=15)
        elif tipo == "revaloracion":
            return hora in ["10:00", "11:00"]
        elif tipo == "post-mamografia":
            return hora_en_franjas(hora, [("12:30", "13:30")], step_minutes=15)

    elif dia in ["miércoles", "viernes"]:
        if tipo == "prioritaria":
            return hora_en_franjas(hora, [("09:30", "10:45"), ("11:30", "14:00")], step_minutes=15)

    return False

### BAG guiada por ecografía

In [ ]:
def es_horario_bag_ecografia(hora: str, dia: str) -> bool:
    """
    BAG por ecografía:
    - Solo martes y jueves
    - Citas puntuales: 09:30–10:00, 10:30–11:00, 11:30–12:00
    """
    dia = dia.lower()
    if dia not in ["martes", "jueves"]:
        return False
    return hora in ["09:30", "10:30", "11:30"]

### PAAF guiada por ecografía

In [ ]:
def es_horario_paaf_ecografia(hora: str, dia: str) -> bool:
    """
    PAAF por ecografía:
    - Solo martes y jueves
    - Citas puntuales: 09:30–10:00, 10:30–11:00, 11:30–12:00 (mismas que BAG)
    """
    dia = dia.lower()
    if dia not in ["martes", "jueves"]:
        return False
    return hora in ["09:30", "10:30", "11:30"]

## RESONANCIA MAGNÉTICA

In [ ]:
def es_horario_resonancia_magnetica(hora: str) -> bool:
    """Resonancia magnética los lunes: 4 franjas de 1 hora."""
    franjas_rm = [
        ("08:45", "09:45"),
        ("10:45", "11:45"),
        ("12:45", "13:45"),
        ("13:00", "14:00"),
    ]
    return hora_en_franjas(hora, franjas_rm, step_minutes=15)

# CALENDARIOS

## Calendario base semanal

In [ ]:
def generar_calendario_base(fecha_inicio: str, dias: int) -> dict:

    calendario = {}
    formato = "%Y-%m-%d"
    fecha = datetime.strptime(fecha_inicio, formato)

    for _ in range(dias):
        dia_str = fecha.strftime(formato)
        calendario[dia_str] = {}
        hora = datetime.strptime("08:00", "%H:%M")
        while hora < datetime.strptime("14:00", "%H:%M"):
            calendario[dia_str][hora.strftime("%H:%M")] = None
            hora += timedelta(minutes=15)
        fecha += timedelta(days=1)

    return calendario

## Calendario de cada sala.

In [ ]:
def mostrar_calendario(calendario: dict, sala: str):
    """
    Muestra el calendario de una sala de forma legible.
    """
    print(f"\n CALENDARIO DE {sala.upper()}")
    for fecha in sorted(calendario.keys()):
        print(f"\n📅 {fecha}")
        horas_ordenadas = sorted(calendario[fecha].items())
        for hora, estado in horas_ordenadas:
            estado_str = "Libre" if estado is None else f"Ocupado: {estado}"
            print(f"  🕒 {hora} - {estado_str}")
    print("\n" + "="*40 + "\n")

calendarios_por_sala = {
    "S7_Mamografia": generar_calendario_base("2025-07-07", 7),
    "S7_Ecografia": generar_calendario_base("2025-07-07", 7),
    "Resonancia": generar_calendario_base("2025-07-07", 7)
}

In [ ]:
for sala, calendario in calendarios_por_sala.items():
    mostrar_calendario(calendario, sala)


 CALENDARIO DE S7_MAMOGRAFIA

📅 2025-07-07
  🕒 08:00 - Libre
  🕒 08:15 - Libre
  🕒 08:30 - Libre
  🕒 08:45 - Libre
  🕒 09:00 - Libre
  🕒 09:15 - Libre
  🕒 09:30 - Libre
  🕒 09:45 - Libre
  🕒 10:00 - Libre
  🕒 10:15 - Libre
  🕒 10:30 - Libre
  🕒 10:45 - Libre
  🕒 11:00 - Libre
  🕒 11:15 - Libre
  🕒 11:30 - Libre
  🕒 11:45 - Libre
  🕒 12:00 - Libre
  🕒 12:15 - Libre
  🕒 12:30 - Libre
  🕒 12:45 - Libre
  🕒 13:00 - Libre
  🕒 13:15 - Libre
  🕒 13:30 - Libre
  🕒 13:45 - Libre

📅 2025-07-08
  🕒 08:00 - Libre
  🕒 08:15 - Libre
  🕒 08:30 - Libre
  🕒 08:45 - Libre
  🕒 09:00 - Libre
  🕒 09:15 - Libre
  🕒 09:30 - Libre
  🕒 09:45 - Libre
  🕒 10:00 - Libre
  🕒 10:15 - Libre
  🕒 10:30 - Libre
  🕒 10:45 - Libre
  🕒 11:00 - Libre
  🕒 11:15 - Libre
  🕒 11:30 - Libre
  🕒 11:45 - Libre
  🕒 12:00 - Libre
  🕒 12:15 - Libre
  🕒 12:30 - Libre
  🕒 12:45 - Libre
  🕒 13:00 - Libre
  🕒 13:15 - Libre
  🕒 13:30 - Libre
  🕒 13:45 - Libre

📅 2025-07-09
  🕒 08:00 - Libre
  🕒 08:15 - Libre
  🕒 08:30 - Libre
  🕒 08:45 

#**FUNCIONES DE ASIGNACIÓN AUTOMÁTICA DE CITAS**


## SALA 7. MAMOGRAFÍA

In [ ]:
def obtener_dia_semana(fecha_str: str) -> str:
    dias = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]
    formato = "%Y-%m-%d"
    fecha_obj = datetime.strptime(fecha_str, formato)
    return dias[fecha_obj.weekday()]

### BAV por exterotaxia

In [ ]:
def asignar_cita_bav_por_etx(calendario: dict, paciente_id: str, nombre: str) -> tuple[str, str] | None:
    dias_semana = {
        0: "lunes",
        4: "viernes",
    }

    for fecha in sorted(calendario.keys()):
        dia_semana = datetime.strptime(fecha, "%Y-%m-%d").weekday() #Así consideramos el día de la semana
        if dia_semana not in dias_semana:
            continue

        nombre_dia = dias_semana[dia_semana]

        for hora in calendario[fecha]:
            if calendario[fecha][hora] is None and es_horario_bav_por_etx(hora, nombre_dia):
                calendario[fecha][hora] = {
                    "estado": "ocupado",
                    "paciente": {
                        "id": paciente_id,
                        "nombre": nombre
                    },
                    "prueba": "BAV por ETX"
                }
                return fecha, hora

    return None #Cuando no encuentre un hueco


### Arpón por esterotaxia

In [ ]:
def asignar_cita_arpon(calendario: dict, paciente_id: str, nombre: str) -> tuple[str, str] | None:
    """
    Asigna automáticamente una cita de arpón:
    - Solo martes y jueves
    - Horarios: 08:30 y 09:15 (45 min)
    """
    for fecha, bloques in calendario.items():
        dia_semana = obtener_dia_semana(fecha)
        for hora, datos in bloques.items():
            if (
                es_horario_arpon(hora, dia_semana)
                and datos is None
            ):

                bloques[hora] = {
                    "estado": "ocupado",
                    "prueba": "arpon",
                    "paciente": {"id": paciente_id, "nombre": nombre}
                }
                return fecha, hora
    return None, None

### Tomografía / Mamografía

In [ ]:
def asignar_cita_tx_mamografia(calendario: dict, paciente_id: str, nombre: str) -> tuple[str, str] | tuple[None, None]:

    for fecha, horas in calendario.items():
        dia_semana = obtener_dia_semana(fecha)

        for hora, info in horas.items():
            if info is not None and info["estado"] == "ocupado":
                continue

            # Evaluamos si ya hay una BAV o Arpón en ese día
            ocupado_bav = any(h is not None and h.get("prueba") == "bav_por_etx" for h in horas.values())
            ocupado_arpon = any(h is not None and h.get("prueba") == "arpon" for h in horas.values())

            if es_horario_tx_mamografia(hora, dia_semana, ocupado_bav, ocupado_arpon):
                # Asignamos la cita
                horas[hora] = { # Assign the dictionary to the slot
                    "estado": "ocupado",
                    "prueba": "tx_mamografia",
                    "paciente": {
                        "id": paciente_id,
                        "nombre": nombre
                    }
                }
                return fecha, hora

    return None, None

### Mamografía con contraste

In [ ]:
def asignar_cita_mamografia_con_contraste(calendario: dict, paciente_id: str, paciente_nombre: str) -> tuple[str, str] | tuple[None, None]:
    """
    Asigna una cita para mamografía con contraste:
    - Solo miércoles
    - Citas cada 45 minutos de 08:45 a 13:45
    """
    for fecha, horas in calendario.items():
        dia_semana = obtener_dia_semana(fecha)

        for hora, info in horas.items():
            if info is not None and info["estado"] == "ocupado":
                continue

            if es_horario_mamografia_con_contraste(hora, dia_semana):
                # Asignar cita
                horas[hora] = { # Assign the dictionary to the slot
                    "estado": "ocupado",
                    "prueba": "mamografia_con_contraste",
                    "paciente": {
                        "id": paciente_id,
                        "nombre": paciente_nombre
                    }
                }
                return fecha, hora

    return None, None

## SALA 7. ECOGRAFÍA

### Ecografías diagnósticas:
Hay que especificar el tipo:
- Prioritaria
- Ordinaria
- Revaloración
- Con contraste


In [ ]:
def asignar_cita_ecografia(calendario: dict, tipo: str, paciente_id: str, nombre: str) -> tuple[str, str] | tuple[None, None]:

    for fecha, horas in calendario.items():
        dia_semana = obtener_dia_semana(fecha)

        for hora, info in horas.items():
            if info is not None and info.get("estado") == "ocupado":
                continue

            if es_horario_ecografia_s7(hora, dia_semana, tipo):

                horas[hora] = {
                    "estado": "ocupado",
                    "tipo_prueba": tipo,
                    "paciente": {
                        "id": paciente_id,
                        "nombre": nombre
                    }
                }
                return fecha, hora

    return None, None

### BAG guiada por ecografía

In [ ]:
def asignar_cita_bag_ecografia(calendario: dict, paciente_id: str, nombre: str) -> tuple[str, str] | tuple[None, None]:
    """
    Asigna automáticamente una cita para BAG por ecografía.
    """
    for fecha, horas in calendario.items():
        dia_semana = obtener_dia_semana(fecha)

        for hora, info in horas.items():
            if info is not None and info["estado"] == "ocupado":
                continue

            if es_horario_bag_ecografia(hora, dia_semana):
                horas[hora] = {
                    "estado": "ocupado",
                    "prueba": "bag_ecografia",
                    "paciente": {
                        "id": paciente_id,
                        "nombre": nombre
                    }
                }
                return fecha, hora

    return None, None

### PAAF guiada por ecografía

In [ ]:
def asignar_cita_paaf_ecografia(calendario: dict, paciente_id: str, nombre: str) -> tuple[str, str] | tuple[None, None]:

    for fecha, horas in calendario.items():
        dia_semana = obtener_dia_semana(fecha)

        for hora, info in horas.items():
            if info is not None and info["estado"] == "ocupado":
                continue

            if es_horario_paaf_ecografia(hora, dia_semana):
                horas[hora] = {
                    "estado": "ocupado",
                    "prueba": "paaf_ecografia",
                    "paciente": {
                        "id": paciente_id,
                        "nombre": nombre
                    }
                }
                return fecha, hora

    return None, None

# Sala Resonancia Magnética

In [ ]:
def asignar_cita_resonancia_magnetica(calendario: dict, paciente_id: str, nombre: str) -> tuple[str, str] | tuple[None, None]:
    for fecha, agenda_dia in calendario.items():
        dia_semana = obtener_dia_semana(fecha)
        if dia_semana.lower() != "lunes":
            continue

        for hora, info in agenda_dia.items():
            if (
                (info is None or info.get("estado") != "ocupado")
                and es_horario_resonancia_magnetica(hora)
            ):
                agenda_dia[hora] = {
                    "estado": "ocupado",
                    "paciente": {"id": paciente_id, "nombre": nombre},
                    "prueba": "resonancia_magnetica"
                }
                return fecha, hora

    return None, None

##Diccionario de funciones de asignación

In [ ]:
asignadores_por_prueba = {
    "bav_por_etx": asignar_cita_bav_por_etx,
    "arpon": asignar_cita_arpon,
    "tx_mamografia": asignar_cita_tx_mamografia,
    "mamografia_con_contraste": asignar_cita_mamografia_con_contraste,

    "ecografia": asignar_cita_ecografia,  # con tipo_ecografia
    "bag_ecografia": asignar_cita_bag_ecografia,
    "paaf_ecografia": asignar_cita_paaf_ecografia,

    "resonancia_magnetica": asignar_cita_resonancia_magnetica
}

# **OPTIMIZACIÓN DE LA CITACIÓN POR PRIORIDAD**

In [ ]:
PRIORIDADES_POR_PRUEBA = {
    "tx_mamografia": ["urgente", "preferente", "ordinaria"],
    "mamografia_con_contraste": ["urgente", "preferente", "ordinaria"],
    "bav_por_etx": ["urgente"],
    "arpon": ["urgente"],
    "ecografia_prioritaria": ["urgente", "preferente"],
    "ecografia_ordinaria": ["ordinaria"],
    "ecografia_revaloracion": ["urgente", "preferente"],
    "ecografia_con_contraste": ["urgente", "preferente"],
    "bag_ecografia": ["urgente", "preferente", "ordinaria"],
    "paaf_ecografia": ["urgente", "preferente", "ordinaria"],
    "bav_rm": ["urgente"],
    "resonancia_magnetica": ["urgente", "preferente", "ordinaria"],
}

PUNTOS_PRIORIDAD = {
    "urgente": 1,
    "preferente": 2,
    "ordinaria": 3
}

### Diccionario de la sala en la que se realiza cada prueba

In [ ]:
sala_por_prueba = {
    # Sala 7 - Mamografía
    "bav_por_etx": "S7_Mamografia",
    "arpon": "S7_Mamografia",
    "tx_mamografia": "S7_Mamografia",
    "mamografia_con_contraste": "S7_Mamografia",

    # Sala 7 - Ecografía
    "ecografia": "S7_Ecografia",  # Se requiere tipo_ecografia adicional
    "bag_ecografia": "S7_Ecografia",
    "paaf_ecografia": "S7_Ecografia",

    # Sala de Resonancia Magnética
    "resonancia_magnetica": "Resonancia"
}

In [ ]:
from functools import partial

reglas_clinicas_por_prueba = {
    "ecografia": lambda hora, dia, tipo: es_horario_ecografia_s7(hora, dia, tipo),
    "bag_ecografia": lambda hora, dia: es_horario_bag_ecografia(hora, dia),
    "paaf_ecografia": lambda hora, dia: es_horario_paaf_ecografia(hora, dia),

    "bav_por_etx": es_horario_bav_por_etx,
    "arpon": es_horario_arpon,
    "tx_mamografia": es_horario_tx_mamografia,
    "mamografia_con_contraste": es_horario_mamografia_con_contraste,

    "resonancia_magnetica": lambda hora, dia: es_horario_resonancia_magnetica(hora),
}

### Lista de espera de los pacientes

In [ ]:
lista_espera = []

In [ ]:
def añadir_a_lista_espera(
    tipo_prueba: str,
    paciente_id: str,
    nombre: str,
    prioridad: str,
    tipo_ecografia: str = None
):
    paciente = {
        "tipo_prueba": tipo_prueba,
        "tipo_ecografia": tipo_ecografia,
        "id": paciente_id,
        "nombre": nombre,
        "prioridad": prioridad
    }
    lista_espera.append(paciente)
    print(f"Paciente {nombre} añadido a la lista de espera ({prioridad})")

In [ ]:
añadir_a_lista_espera("ecografia", "123456", "Lucía García", "urgente", tipo_ecografia="revaloracion")
añadir_a_lista_espera("mamografia_con_contraste", "4561234", "Ana Pérez", "ordinaria")

Paciente Lucía García añadido a la lista de espera (urgente)
Paciente Ana Pérez añadido a la lista de espera (ordinaria)


In [ ]:
añadir_a_lista_espera("ecografia", "12345678", "Lucía García", "preferente", tipo_ecografia="revaloracion")
añadir_a_lista_espera("paaf_ecografia", "1234567", "Milagros Gómez", "preferente")
añadir_a_lista_espera("bag_ecografia", "1234567", "Blanca Gómez", "urgente")
añadir_a_lista_espera("ecografia", "0000000", "Gloria Pérez", "urgente", tipo_ecografia="contraste")
añadir_a_lista_espera("ecografia", "98765432", "Laura López", "prioritaria", tipo_ecografia="prioritaria")
añadir_a_lista_espera("mamografia_con_contraste", "98765432", "Mireia Lozano", "prioritaria")
añadir_a_lista_espera("arpon", "1234567", "Paula Gómez", "urgente")
añadir_a_lista_espera("arpon", "1234567", "Andrea Campos", "urgente")
añadir_a_lista_espera("ecografia", "98765432", "Carlos Martínez", "preferente", tipo_ecografia="revaloracion")
añadir_a_lista_espera("tx_mamografia", "12345678", "María López", "ordinaria")
añadir_a_lista_espera("resonancia_magnetica", "12345678", "Lucía García", "urgente")
añadir_a_lista_espera("tx_mamografia", "12345678", "Marta López", "ordinaria")

Paciente Lucía García añadido a la lista de espera (preferente)
Paciente Milagros Gómez añadido a la lista de espera (preferente)
Paciente Blanca Gómez añadido a la lista de espera (urgente)
Paciente Gloria Pérez añadido a la lista de espera (urgente)
Paciente Laura López añadido a la lista de espera (prioritaria)
Paciente Mireia Lozano añadido a la lista de espera (prioritaria)
Paciente Paula Gómez añadido a la lista de espera (urgente)
Paciente Andrea Campos añadido a la lista de espera (urgente)
Paciente Carlos Martínez añadido a la lista de espera (preferente)
Paciente María López añadido a la lista de espera (ordinaria)
Paciente Lucía García añadido a la lista de espera (urgente)
Paciente Marta López añadido a la lista de espera (ordinaria)


### Función de optimización por prioridad

In [ ]:
def optimizar_desde_lista_espera(
    calendario_por_sala: dict,
    reglas_clinicas_por_prueba: dict,
    sala_por_prueba: dict
) -> list[tuple]:

    # 1. Ordenar los pacientes según nivel de prioridad
    lista_espera.sort(key=lambda p: PUNTOS_PRIORIDAD.get(p.get("prioridad", "ordinaria"), 99))

    asignaciones = []

    for paciente in lista_espera:
        tipo = paciente["tipo_prueba"]
        prioridad = paciente.get("prioridad")
        regla = reglas_clinicas_por_prueba.get(tipo)
        sala = sala_por_prueba.get(tipo)

        if not regla or not sala:
            print(f"Faltan datos para asignar {paciente['nombre']} ({tipo})")
            continue

        calendario = calendario_por_sala.get(sala)
        if not calendario:
            print(f"No hay calendario para la sala {sala}")
            continue

        asignado = False

        for fecha in sorted(calendario):
            dia = obtener_dia_semana(fecha)

            for hora in sorted(calendario[fecha]):
                slot = calendario[fecha][hora]
                if slot is None and regla(hora, dia):
                    calendario[fecha][hora] = {
                        "estado": "ocupado",
                        "tipo_prueba": tipo,
                        "paciente": {
                            "id": paciente["paciente_id"],
                            "nombre": paciente["nombre"],
                            "prioridad": prioridad
                        }
                    }
                    asignaciones.append((paciente["paciente_id"], paciente["nombre"], tipo, prioridad, fecha, hora))
                    asignado = True
                    break

            if asignado:
                break

        if not asignado:
            print(f"No se encontró hueco para {paciente['nombre']} ({tipo})")

    # Limpiar la lista una vez optimizado
    lista_espera.clear()
    return asignaciones

### Función de asignación de pacientes desde la lista de espera

In [ ]:
def asignar_desde_lista_espera():
    for paciente in lista_espera:
        tipo_prueba = paciente["tipo_prueba"]
        paciente_id = paciente["id"]
        nombre = paciente["nombre"]
        prioridad = paciente["prioridad"]
        tipo_ecografia = paciente.get("tipo_ecografia")

        if tipo_prueba == "ecografia":
            if not tipo_ecografia:
                print(f"{nombre}: Debes especificar el tipo de ecografía.")
                continue
            tipo_final = "ecografia"
        else:
            tipo_final = tipo_prueba

        regla = reglas_clinicas_por_prueba.get(tipo_final)
        sala = sala_por_prueba.get(tipo_final)

        if not regla or not sala:
            print(f"{nombre}: No hay reglas o sala definida para la prueba '{tipo_final}'")
            continue

        calendario = calendarios_por_sala.get(sala)
        if not calendario:
            print(f"{nombre}: No hay calendario disponible para la sala '{sala}'")
            continue

        asignado = False

        for fecha in sorted(calendario):
            dia = obtener_dia_semana(fecha)

            for hora in sorted(calendario[fecha]):
                slot = calendario[fecha][hora]

                if slot is None:
                    if tipo_prueba == "ecografia":
                        es_valido = regla(hora, dia, tipo_ecografia)
                    else:
                        es_valido = regla(hora, dia)

                    if es_valido:
                        calendario[fecha][hora] = {
                            "estado": "ocupado",
                            "tipo_prueba": tipo_final,
                            "paciente": {
                                "id": paciente_id,
                                "nombre": nombre,
                                "prioridad": prioridad
                            }
                        }
                        print(f"{nombre} ({prioridad}) asignado a {fecha} a las {hora} en {sala}")
                        asignado = True
                        break
            if asignado:
                break

        if not asignado:
            print(f"No hay huecos disponibles para {nombre} ({tipo_prueba})")



In [ ]:
asignar_desde_lista_espera()

Lucía García (urgente) asignado a 2025-07-07 a las 09:00 en S7_Ecografia
Ana Pérez (ordinaria) asignado a 2025-07-09 a las 08:45 en S7_Mamografia
Lucía García (preferente) asignado a 2025-07-07 a las 09:30 en S7_Ecografia
Milagros Gómez (preferente) asignado a 2025-07-08 a las 09:30 en S7_Ecografia
Blanca Gómez (urgente) asignado a 2025-07-08 a las 10:30 en S7_Ecografia
Gloria Pérez (urgente) asignado a 2025-07-07 a las 10:00 en S7_Ecografia
Laura López (prioritaria) asignado a 2025-07-08 a las 09:00 en S7_Ecografia
Mireia Lozano (prioritaria) asignado a 2025-07-09 a las 09:30 en S7_Mamografia
Paula Gómez (urgente) asignado a 2025-07-08 a las 08:30 en S7_Mamografia
Andrea Campos (urgente) asignado a 2025-07-08 a las 09:15 en S7_Mamografia
Carlos Martínez (preferente) asignado a 2025-07-07 a las 10:30 en S7_Ecografia
María López (ordinaria) asignado a 2025-07-07 a las 08:30 en S7_Mamografia
Lucía García (urgente) asignado a 2025-07-07 a las 08:45 en Resonancia
Marta López (ordinaria) as

In [ ]:
def mostrar_calendario(calendario: dict, titulo: str = "Calendario"):
    print(f"\n=== {titulo} ===")
    for fecha in sorted(calendario):
        print(f"\n📅 {fecha}")
        horas = calendario[fecha]
        for hora in sorted(horas):
            info = horas[hora]
            if info is None:
                estado = "🟩 LIBRE"
            else:
                estado = "🟥 OCUPADO"
                tipo = info.get("tipo_prueba") or info.get("prueba") or "N/A"
                paciente = info.get("paciente", {})
                nombre = paciente.get("nombre")
                estado += f" | {tipo} | {nombre}"
            print(f"  ⏰ {hora}: {estado}")

In [ ]:
mostrar_calendario(calendarios_por_sala["S7_Mamografia"])


=== Calendario ===

📅 2025-07-07
  ⏰ 08:00: 🟩 LIBRE
  ⏰ 08:15: 🟩 LIBRE
  ⏰ 08:30: 🟥 OCUPADO | tx_mamografia | María López
  ⏰ 08:45: 🟥 OCUPADO | tx_mamografia | Marta López
  ⏰ 09:00: 🟩 LIBRE
  ⏰ 09:15: 🟩 LIBRE
  ⏰ 09:30: 🟩 LIBRE
  ⏰ 09:45: 🟩 LIBRE
  ⏰ 10:00: 🟩 LIBRE
  ⏰ 10:15: 🟩 LIBRE
  ⏰ 10:30: 🟩 LIBRE
  ⏰ 10:45: 🟩 LIBRE
  ⏰ 11:00: 🟩 LIBRE
  ⏰ 11:15: 🟩 LIBRE
  ⏰ 11:30: 🟩 LIBRE
  ⏰ 11:45: 🟩 LIBRE
  ⏰ 12:00: 🟩 LIBRE
  ⏰ 12:15: 🟩 LIBRE
  ⏰ 12:30: 🟩 LIBRE
  ⏰ 12:45: 🟩 LIBRE
  ⏰ 13:00: 🟩 LIBRE
  ⏰ 13:15: 🟩 LIBRE
  ⏰ 13:30: 🟩 LIBRE
  ⏰ 13:45: 🟩 LIBRE

📅 2025-07-08
  ⏰ 08:00: 🟩 LIBRE
  ⏰ 08:15: 🟩 LIBRE
  ⏰ 08:30: 🟥 OCUPADO | arpon | Paula Gómez
  ⏰ 08:45: 🟩 LIBRE
  ⏰ 09:00: 🟩 LIBRE
  ⏰ 09:15: 🟥 OCUPADO | arpon | Andrea Campos
  ⏰ 09:30: 🟩 LIBRE
  ⏰ 09:45: 🟩 LIBRE
  ⏰ 10:00: 🟩 LIBRE
  ⏰ 10:15: 🟩 LIBRE
  ⏰ 10:30: 🟩 LIBRE
  ⏰ 10:45: 🟩 LIBRE
  ⏰ 11:00: 🟩 LIBRE
  ⏰ 11:15: 🟩 LIBRE
  ⏰ 11:30: 🟩 LIBRE
  ⏰ 11:45: 🟩 LIBRE
  ⏰ 12:00: 🟩 LIBRE
  ⏰ 12:15: 🟩 LIBRE
  ⏰ 12:30: 🟩 LIBRE
  ⏰ 12:45: 🟩 LIBRE
  